Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.872780
Minibatch accuracy: 12.5%
Validation accuracy: 9.8%
Minibatch loss at step 50: 1.828506
Minibatch accuracy: 43.8%
Validation accuracy: 44.6%
Minibatch loss at step 100: 1.138049
Minibatch accuracy: 62.5%
Validation accuracy: 71.4%
Minibatch loss at step 150: 0.665964
Minibatch accuracy: 81.2%
Validation accuracy: 74.1%
Minibatch loss at step 200: 0.911139
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 250: 1.160255
Minibatch accuracy: 62.5%
Validation accuracy: 77.9%
Minibatch loss at step 300: 0.343764
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.394178
Minibatch accuracy: 93.8%
Validation accuracy: 77.0%
Minibatch loss at step 400: 0.217088
Minibatch accuracy: 93.8%
Validation accuracy: 79.9%
Minibatch loss at step 450: 0.817980
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.683137
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.811460
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.893216
Minibatch accuracy: 25.0%
Validation accuracy: 36.4%
Minibatch loss at step 100: 1.016899
Minibatch accuracy: 50.0%
Validation accuracy: 60.4%
Minibatch loss at step 150: 0.555304
Minibatch accuracy: 75.0%
Validation accuracy: 71.7%
Minibatch loss at step 200: 0.989024
Minibatch accuracy: 75.0%
Validation accuracy: 77.3%
Minibatch loss at step 250: 1.095820
Minibatch accuracy: 68.8%
Validation accuracy: 75.7%
Minibatch loss at step 300: 0.362549
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 350: 0.541274
Minibatch accuracy: 93.8%
Validation accuracy: 75.7%
Minibatch loss at step 400: 0.302812
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.694500
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 500: 0.836988
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

# This reached 90.2%

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  layer5_weights = tf.Variable(tf.truncated_normal([32, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  # Model.
  def model(data, dropout_rate=1.):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.nn.dropout( tf.matmul(reshape, layer3_weights) + layer3_biases, dropout_rate))
#     hidden = tf.nn.dropout(hidden, dropout_rate)
    
#     hidden2 = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.6)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [41]:
num_steps = 1001

''' Try playing with steps, learning rate decay, initialization of variable - what is possible to do? change optimizer to Adam '''
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.029192
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.317182
Minibatch accuracy: 0.0%
Validation accuracy: 11.8%
Minibatch loss at step 100: 2.147567
Minibatch accuracy: 6.2%
Validation accuracy: 29.3%
Minibatch loss at step 150: 1.455296
Minibatch accuracy: 50.0%
Validation accuracy: 56.2%
Minibatch loss at step 200: 1.348866
Minibatch accuracy: 43.8%
Validation accuracy: 64.8%
Minibatch loss at step 250: 1.517313
Minibatch accuracy: 50.0%
Validation accuracy: 70.7%
Minibatch loss at step 300: 0.803217
Minibatch accuracy: 62.5%
Validation accuracy: 78.3%
Minibatch loss at step 350: 0.608284
Minibatch accuracy: 87.5%
Validation accuracy: 76.8%
Minibatch loss at step 400: 0.379667
Minibatch accuracy: 93.8%
Validation accuracy: 77.9%
Minibatch loss at step 450: 0.933457
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 500: 1.206052
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Mini

# Just increased to 4000 steps
 -> reached 92.7

# Here made a pyramid -
First layer makes a depth of 8
2nd layer makes a depth of 16
3rd layer is a fully connected layer with very low dropout
4th layer is a fully connected layer that changes to the labels shape
400 steps
disabled dropout
-> Resulted in 93.3%

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
def k_max_pooling(last_layer, k):
    return tf.nn.max_pool(last_layer, [1,k,k,1], [1,k,k,1], padding='SAME')
def conv_layer(last_layer, weights, biases):
    return tf.nn.relu( \
                      tf.nn.conv2d(last_layer, weights, [1, 1, 1, 1], padding='SAME') + \
                      biases)

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # First layer of convolution - changes depth from the input's 1 monochrome to 8
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth // 2], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth // 2]))
  # Second layer - convolution that changes the pyramid depth to 16
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth//2, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  # Fully connected layer that changes from the conv. output shape to arbitrary number
  # (on each step we divided the width/height by 2, totalling by 4. // is like cpp div. multiplying width//4, height//4 and depth)
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  # Fully connected layer that changes from the arbitrary number to the classification labels count  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def model(data, dropout_rate=1.):
    # we use pooling so the strides are of 1, and the pooling will be for patches of size 2
    layer_1 = conv_layer(data, layer1_weights, layer1_biases)
    layer_1 = k_max_pooling(layer_1, 2)
    
    # We can use here the regular strides
    layer_2 = conv_layer(layer_1, layer2_weights, layer2_biases)
    layer_2 = k_max_pooling(layer_2, 2)
    
    # Reshaping the last layer - combining the 3d matrix to a vector
    shape = layer_2.get_shape().as_list()
    reshape = tf.reshape(layer_2, [shape[0], shape[1] * shape[2] * shape[3]])
    layer_3 = tf.nn.relu( \
                tf.nn.dropout( \
                    tf.matmul(reshape, layer3_weights) + layer3_biases, \
                              dropout_rate))

    # We are recovering from the dropout here and switching to the output size
    layer_4 = tf.matmul(layer_3, layer4_weights) + layer4_biases
    return layer_4
  
  # Training computation. without dropout
  logits = model(tf_train_dataset, 1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #learning_rate = tf.train.polynomial_decay(0.99, global_step, num_steps - 1, power=0.5)
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 4001

''' Try playing with steps, learning rate decay, initialization of variable - what is possible to do? change optimizer to Adam '''
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.041953
Minibatch accuracy: 12.5%
Validation accuracy: 9.8%
Minibatch loss at step 50: 2.320145
Minibatch accuracy: 12.5%
Validation accuracy: 12.3%
Minibatch loss at step 100: 2.199592
Minibatch accuracy: 31.2%
Validation accuracy: 23.0%
Minibatch loss at step 150: 1.302726
Minibatch accuracy: 43.8%
Validation accuracy: 46.8%
Minibatch loss at step 200: 1.110744
Minibatch accuracy: 68.8%
Validation accuracy: 63.5%
Minibatch loss at step 250: 1.284818
Minibatch accuracy: 68.8%
Validation accuracy: 72.9%
Minibatch loss at step 300: 0.344343
Minibatch accuracy: 75.0%
Validation accuracy: 76.1%
Minibatch loss at step 350: 0.480033
Minibatch accuracy: 93.8%
Validation accuracy: 76.1%
Minibatch loss at step 400: 0.414833
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 450: 0.995881
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%
Minibatch loss at step 500: 0.732005
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
Mi

# Adding the dropout
dropout value 0.75
batch size increased to 128
on the 4000 step - 94.2%
on the 5000 step (sorry) - 94.3%

In [17]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
def k_max_pooling(last_layer, k):
    return tf.nn.max_pool(last_layer, [1,k,k,1], [1,k,k,1], padding='SAME')
def conv_layer(last_layer, weights, biases):
    return tf.nn.relu( \
                      tf.nn.conv2d(last_layer, weights, [1, 1, 1, 1], padding='SAME') + \
                      biases)

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # First layer of convolution - changes depth from the input's 1 monochrome to 8
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth // 2], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth // 2]))
  # Second layer - convolution that changes the pyramid depth to 16
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth//2, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  # Fully connected layer that changes from the conv. output shape to arbitrary number
  # (on each step we divided the width/height by 2, totalling by 4. // is like cpp div. multiplying width//4, height//4 and depth)
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  # Fully connected layer that changes from the arbitrary number to the classification labels count  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def model(data, dropout_rate=1.):
    # we use pooling so the strides are of 1, and the pooling will be for patches of size 2
    layer_1 = conv_layer(data, layer1_weights, layer1_biases)
    layer_1 = k_max_pooling(layer_1, 2)
    
    # We can use here the regular strides
    layer_2 = conv_layer(layer_1, layer2_weights, layer2_biases)
    layer_2 = k_max_pooling(layer_2, 2)
    
    # Reshaping the last layer - combining the 3d matrix to a vector
    shape = layer_2.get_shape().as_list()
    reshape = tf.reshape(layer_2, [shape[0], shape[1] * shape[2] * shape[3]])
    layer_3 = tf.nn.relu( \
                tf.nn.dropout( \
                    tf.matmul(reshape, layer3_weights) + layer3_biases, \
                              dropout_rate))

    # We are recovering from the dropout here and switching to the output size
    layer_4 = tf.matmul(layer_3, layer4_weights) + layer4_biases
    return layer_4
  
  # Training computation. without dropout
  logits = model(tf_train_dataset, 0.75)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #learning_rate = tf.train.polynomial_decay(0.99, global_step, num_steps - 1, power=0.5)
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 5001

''' Try playing with steps, learning rate decay, initialization of variable - what is possible to do? change optimizer to Adam '''
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    if (step % 1000 == 0):
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  

Initialized
Minibatch loss at step 0: 3.213450
Minibatch accuracy: 9.4%
Validation accuracy: 9.8%
Test accuracy: 9.9%
Minibatch loss at step 50: 1.440960
Minibatch accuracy: 57.0%
Validation accuracy: 60.3%
Minibatch loss at step 100: 0.999313
Minibatch accuracy: 68.0%
Validation accuracy: 75.0%
Minibatch loss at step 150: 1.065335
Minibatch accuracy: 68.0%
Validation accuracy: 79.2%
Minibatch loss at step 200: 0.782701
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 250: 0.635612
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Minibatch loss at step 300: 0.697543
Minibatch accuracy: 77.3%
Validation accuracy: 81.6%
Minibatch loss at step 350: 0.520820
Minibatch accuracy: 83.6%
Validation accuracy: 82.2%
Minibatch loss at step 400: 0.751683
Minibatch accuracy: 75.8%
Validation accuracy: 83.1%
Minibatch loss at step 450: 0.576246
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 500: 0.461478
Minibatch accuracy: 83.6%
Validation

Validation accuracy: 87.8%
Minibatch loss at step 4500: 0.400525
Minibatch accuracy: 86.7%
Validation accuracy: 87.8%
Minibatch loss at step 4550: 0.478323
Minibatch accuracy: 85.9%
Validation accuracy: 87.8%
Minibatch loss at step 4600: 0.417476
Minibatch accuracy: 85.2%
Validation accuracy: 87.8%
Minibatch loss at step 4650: 0.641014
Minibatch accuracy: 80.5%
Validation accuracy: 88.0%
Minibatch loss at step 4700: 0.499384
Minibatch accuracy: 82.8%
Validation accuracy: 87.8%
Minibatch loss at step 4750: 0.292533
Minibatch accuracy: 89.1%
Validation accuracy: 88.0%
Minibatch loss at step 4800: 0.425577
Minibatch accuracy: 85.9%
Validation accuracy: 88.0%
Minibatch loss at step 4850: 0.531058
Minibatch accuracy: 84.4%
Validation accuracy: 88.0%
Minibatch loss at step 4900: 0.273014
Minibatch accuracy: 92.2%
Validation accuracy: 87.8%
Minibatch loss at step 4950: 0.488701
Minibatch accuracy: 85.9%
Validation accuracy: 88.1%
Minibatch loss at step 5000: 0.391507
Minibatch accuracy: 86.7%

# Adding linear decay to the learning rate
this will show the evil loss not to rise.
4000 steps - 93.6%

Doubled the depth to 32
500 steps - 87.5%

In [26]:
num_steps = 500
batch_size = 128
patch_size = 5
depth = 32
num_hidden = 1024

graph = tf.Graph()
def k_max_pooling(last_layer, k):
    return tf.nn.max_pool(last_layer, [1,k,k,1], [1,k,k,1], padding='SAME')
def conv_layer(last_layer, weights, biases):
    return tf.nn.relu( \
                      tf.nn.conv2d(last_layer, weights, [1, 1, 1, 1], padding='SAME') + \
                      biases)

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # First layer of convolution - changes depth from the input's 1 monochrome to 8
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth // 2], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth // 2]))
  # Second layer - convolution that changes the pyramid depth to 16
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth//2, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  # Fully connected layer that changes from the conv. output shape to arbitrary number
  # (on each step we divided the width/height by 2, totalling by 4. // is like cpp div. multiplying width//4, height//4 and depth)
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  # Fully connected layer that changes from the arbitrary number to the classification labels count  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def model(data, dropout_rate=1.):
    # we use pooling so the strides are of 1, and the pooling will be for patches of size 2
    layer_1 = conv_layer(data, layer1_weights, layer1_biases)
    layer_1 = k_max_pooling(layer_1, 2)
    
    # We can use here the regular strides
    layer_2 = conv_layer(layer_1, layer2_weights, layer2_biases)
    layer_2 = k_max_pooling(layer_2, 2)
    
    # Reshaping the last layer - combining the 3d matrix to a vector
    shape = layer_2.get_shape().as_list()
    reshape = tf.reshape(layer_2, [shape[0], shape[1] * shape[2] * shape[3]])
    layer_3 = tf.nn.relu( \
                tf.nn.dropout( \
                    tf.matmul(reshape, layer3_weights) + layer3_biases, \
                              dropout_rate))

    # We are recovering from the dropout here and switching to the output size
    layer_4 = tf.matmul(layer_3, layer4_weights) + layer4_biases
    return layer_4
  
  # Training computation. without dropout
  logits = model(tf_train_dataset, 0.75)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.polynomial_decay(0.05, global_step, num_steps - 1) #0.05 to 0.0001
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [27]:
''' Try playing with steps, learning rate decay, initialization of variable - what is possible to do? change optimizer to Adam '''
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    if (step % 1000 == 0):
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  

Initialized
Minibatch loss at step 0: 14.518277
Minibatch accuracy: 10.9%
Validation accuracy: 13.1%
Test accuracy: 13.2%
Minibatch loss at step 50: 9.131517
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 8.203741
Minibatch accuracy: 7.8%
Validation accuracy: 10.1%
Minibatch loss at step 150: 3.009784
Minibatch accuracy: 30.5%
Validation accuracy: 39.8%
Minibatch loss at step 200: 1.907857
Minibatch accuracy: 53.9%
Validation accuracy: 61.8%
Minibatch loss at step 250: 0.900675
Minibatch accuracy: 74.2%
Validation accuracy: 72.8%
Minibatch loss at step 300: 1.162875
Minibatch accuracy: 67.2%
Validation accuracy: 75.3%
Minibatch loss at step 350: 0.857185
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 400: 1.188458
Minibatch accuracy: 69.5%
Validation accuracy: 78.4%
Minibatch loss at step 450: 0.910190
Minibatch accuracy: 74.2%
Validation accuracy: 79.8%
Test accuracy: 87.5%


# Switched to AdamOptimizer
couldn't get results. eh

In [43]:
num_steps = 500
batch_size = 64
patch_size = 5
depth = 32
num_hidden = 1024

graph = tf.Graph()
def k_max_pooling(last_layer, k):
    return tf.nn.max_pool(last_layer, [1,k,k,1], [1,k,k,1], padding='SAME')
def conv_layer(last_layer, weights, biases):
    return tf.nn.relu( \
                      tf.nn.conv2d(last_layer, weights, [1, 1, 1, 1], padding='SAME') + \
                      biases)

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # First layer of convolution - changes depth from the input's 1 monochrome to 8
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth // 2], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth // 2]))
  # Second layer - convolution that changes the pyramid depth to 16
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth//2, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  # Fully connected layer that changes from the conv. output shape to arbitrary number
  # (on each step we divided the width/height by 2, totalling by 4. // is like cpp div. multiplying width//4, height//4 and depth)
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  # Fully connected layer that changes from the arbitrary number to the classification labels count  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def model(data, dropout_rate=1.):
    # we use pooling so the strides are of 1, and the pooling will be for patches of size 2
    layer_1 = conv_layer(data, layer1_weights, layer1_biases)
    layer_1 = k_max_pooling(layer_1, 2)
    
    # We can use here the regular strides
    layer_2 = conv_layer(layer_1, layer2_weights, layer2_biases)
    layer_2 = k_max_pooling(layer_2, 2)
    
    # Reshaping the last layer - combining the 3d matrix to a vector
    shape = layer_2.get_shape().as_list()
    reshape = tf.reshape(layer_2, [shape[0], shape[1] * shape[2] * shape[3]])
    layer_3 = tf.nn.relu( \
                tf.nn.dropout( \
                    tf.matmul(reshape, layer3_weights) + layer3_biases, \
                              dropout_rate))

    # We are recovering from the dropout here and switching to the output size
    layer_4 = tf.matmul(layer_3, layer4_weights) + layer4_biases
    return layer_4
  
  # Training computation. without dropout
  logits = model(tf_train_dataset, 0.75)
  predictions = tf.nn.softmax(logits)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.polynomial_decay(0.05, global_step, num_steps - 1) #0.05 to 0.0001
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  init = tf.global_variables_initializer()


In [44]:
''' Try playing with steps, learning rate decay, initialization of variable - what is possible to do? change optimizer to Adam '''
with tf.Session(graph=graph) as session:
  session.run(init)
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, valid = session.run(
      [optimizer, loss, train_prediction, valid_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid, valid_labels))
    if (step % 1000 == 0):
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  

Initialized
Minibatch loss at step 0: 25.097654
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Test accuracy: 9.7%
Minibatch loss at step 50: 5.124414
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 100: 4.390482
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%


KeyboardInterrupt: 